In [1]:
import jax
import jax.numpy as jnp
import flax.linen as nn

batch_size = 32
feature_dim = 768
layers = 12
rng = jax.random.key(42)
x = jnp.ones((batch_size, feature_dim))

# Mine

In [2]:
class ResidualMLPBlock(nn.Module):
  feature_dim: int = 3
  @nn.compact
  def __call__(self, layer_id, x):
    h = nn.Dense(features=self.feature_dim)(x)
    h = nn.relu(h)
    o = x+h
    return layer_id+1, o
    # out = {}
    # out['layer_id'] = layer_id
    # out['o'] = o
    # return layer_id+1, o, out

class ResidualMLP(nn.Module):
  n_layers: int = 4
  feature_dim: int = 3
  remat_policy: str = "nothing_saveable"

  @nn.compact
  def __call__(self, x):

    block = nn.remat(
      ResidualMLPBlock, 
      prevent_cse=False,
      policy=getattr(jax.checkpoint_policies, self.remat_policy, None),
    )

    ScanMLP = nn.scan(
      block, 
      variable_axes={'params': 0},
      split_rngs={'params': True},
      in_axes=nn.broadcast,
      length=self.n_layers)

    init_layer_id = 0
    final_layer_id, y = ScanMLP(
      feature_dim=self.feature_dim,
    )(init_layer_id, x)
    return final_layer_id, y
  

model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
layer_id, y = model.apply(variables, x)
print(f"variable shapes: {jax.tree.map(lambda x: x.shape, variables)}")
print(f"layer_id: {layer_id}")
print(f"y shape: {y.shape}")

variable shapes: {'params': {'ScanCheckpointResidualMLPBlock_0': {'Dense_0': {'bias': (12, 768), 'kernel': (12, 768, 768)}}}}
layer_id: 12
y shape: (12, 32, 768)


# Big Vision

In [3]:
class ResidualMLPBlock(nn.Module):
  feature_dim: int = 3
  @nn.compact
  def __call__(self, x, deterministic=True):
    out = {}
    h = nn.Dense(features=self.feature_dim)(x)
    h = nn.relu(h)
    o = out['o'] = x+h
    return o, out

class ResidualMLP(nn.Module):
  n_layers: int = 4
  feature_dim: int = 3
  remat_policy: str = "nothing_saveable"

  @nn.compact
  def __call__(self, x, deterministic=True):
    block = nn.remat(
      ResidualMLPBlock, 
      prevent_cse=False,
      static_argnums=(2,),
      policy=getattr(jax.checkpoint_policies, self.remat_policy, None),)
    
    ScanMLP = nn.scan(
      block, 
      variable_axes={'params': 0},
      split_rngs={'params': True},
      in_axes=nn.broadcast,
      length=self.n_layers)
    
    x, scan_out = ScanMLP(
      feature_dim=self.feature_dim
    )(x,deterministic)
    return x, scan_out


model = ResidualMLP(n_layers=layers, feature_dim=feature_dim)
variables = model.init(rng, x)
y, scan_out = model.apply(variables, x, True)
print(f"variable shapes: {jax.tree.map(lambda x: x.shape, variables)}")
print(f"y shape: {y.shape}")
print(f"scan_out shapes: {jax.tree.map(lambda x: x.shape, scan_out)}")

variable shapes: {'params': {'ScanCheckpointResidualMLPBlock_0': {'Dense_0': {'bias': (12, 768), 'kernel': (12, 768, 768)}}}}
y shape: (32, 768)
scan_out shapes: {'o': (12, 32, 768)}
